In [ ]:
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import sys
from scipy.signal import convolve, welch, periodogram
from scipy.fft import fftshift

sys.path.append('../Modules/')
# from plotting import *
from least_squares_models import *
from metrics import *

In [ ]:
data = loadmat("../FOR_COOPERATION/1TR_C20Nc1CD_E20Ne1CD_20250117_5m/1TR_C20Nc1CD_E20Ne1CD_20250117_5m.mat")
fil = loadmat("../FOR_COOPERATION/rx_filter.mat")
# plot_spectrum(data)

In [ ]:
data.keys()

In [ ]:
rxa = data["rxa"][0]
txa = data["txa"][0]
nfa = data["nfa"][0]
pim_ext = data["PIM_EXT"][0]
pim = data["PIM_COND"][0] + data["PIM_EXT"][0]
    
FC_TX = data['BANDS_DL'][0][0][0][0][0] / 10**6
FC_RX = data['BANDS_UL'][0][0][0][0][0] / 10**6
FS = data['Fs'][0][0] / 10**6
PIM_SFT = data['PIM_sft'][0][0] / 10**6
PIM_BW = data['BANDS_TX'][0][0][1][0][0] / 10**6

In [ ]:
def create_model_matrix(x, bias=False):
    basis_function = x * (np.abs(x) ** 2)
    n = x.shape[0]
    gen_mmat = np.empty((n,2), dtype=np.complex_)
    gen_mmat[:,0] = basis_function
    gen_mmat[:,1] = np.ones((n,), dtype=np.complex_)
    mmat = gen_mmat 
    if bias == False:
        mmat = gen_mmat[:, 0].reshape(n,1)
    return mmat
    

def create_filtered_model_matrix(x, bias=False):
    basis_function = x * (np.abs(x) ** 2)

    # to bring amplitude of basis functions to the level of pim
    # this is for simplicity, in real our model should learn this filter
    bf_conv = convolve(basis_function, fil['flt_coeff'].flatten())
    n = bf_conv.shape[0]
    gen_mmat = np.empty((n,2), dtype=np.complex_)
    gen_mmat[:,0] = bf_conv
    gen_mmat[:,1] = np.ones((n,), dtype=np.complex_)
    mmat = gen_mmat 
    if bias == False:
        mmat = gen_mmat[:, 0].reshape(n,1)
    return mmat


def create_shifted_matrix(x, n_pts, n_past, n_after):
    win_len = n_past + n_after + 1
    gen_mmat = np.empty((n_pts,win_len), dtype=np.complex_)
    for i in range(win_len):
        gen_mmat[:,i] = x[i:i+n_pts]
    return gen_mmat


def real_model_matrix(cplx_tr_sig, bias=False):
    x = np.real(cplx_tr_sig)
    n = x.shape[0]
    gen_mmat = np.empty((n,2))
    gen_mmat[:,0] = x * (np.abs(x) ** 2)
    gen_mmat[:,1] = np.ones((n,))
    mmat = gen_mmat 
    if bias == False:
        mmat = gen_mmat[:, 0].reshape(n,1)
    return mmat

### Without filtering

In [ ]:
A = create_model_matrix(txa, bias=True)
print(A.shape)
rhs = rxa
w = np.linalg.inv(A.conj().T @ A) @ A.conj().T @ rhs
print(w)

In [ ]:
pred_pim = A @ w
filt_signal = rhs - pred_pim

In [ ]:
rxa_pow = cal_power(rxa,FS,FC_TX=FC_TX,PIM_SFT=PIM_SFT,PIM_BW=PIM_BW)
filt_pow = cal_power(filt_signal, FS, FC_TX = FC_TX, PIM_SFT = PIM_SFT, PIM_BW = PIM_BW)
perf = 10*np.log10(10**((rxa_pow)/10) - 1) - 10*np.log10(10**((filt_pow)/10) - 1)
print(perf)

In [ ]:
rxa_pow

In [ ]:
10**((rxa_pow)/10)

In [ ]:
rxa_pow_my

In [ ]:
10*np.log10(10**((filt_pow)/10) - 1)

In [ ]:
rxa_pow_my = compute_power(rxa, FS, PIM_SFT, PIM_BW)
filt_pow_my = compute_power(filt_signal, FS, PIM_SFT, PIM_BW)

In [ ]:
10*np.log10((rxa_pow_my - 1) / (filt_pow_my - 1))

In [ ]:
calculate_metrics(rxa, filt_signal, FS, FC_TX, PIM_SFT, PIM_BW)

In [ ]:
initial_power = compute_power(rxa, FS , PIM_SFT, PIM_BW)
filt_power = compute_power(filt_signal,FS, PIM_SFT, PIM_BW)
print(initial_power, filt_power)
calc_perf2(initial_power, filt_power)

In [ ]:
plt.plot(psd_sig)

In [ ]:
plt.plot(fftshift(pxx))

In [ ]:
ax = plt.subplot(1,1,1)
psd_TX,f = ax.psd(txa, Fs = FS, Fc = FC_TX, NFFT = 2048, window = np.kaiser(2048,10), noverlap = 1, pad_to = 2048)
# psd_RX,f = ax.psd(rxa, Fs = FS, Fc = FC_TX, NFFT = 2048, window = np.kaiser(2048,10), noverlap = 1, pad_to = 2048)
# psd_NF,f = ax.psd(pim, Fs = FS, Fc = FC_TX, NFFT = 2048, window = np.kaiser(2048,10), noverlap = 1, pad_to = 2048)
# psd_PIM,f = ax.psd(filt_signal, Fs = FS, Fc = FC_TX, NFFT = 2048, window = np.kaiser(2048,10), noverlap = 1, pad_to = 2048)
ax.set_ylabel(r'PSD, $V^2$/Hz [dB]')
ax.set_xlabel('Frequency, MHz')
ax.set_title('Power spectral density')
plt.show()

In [ ]:
ax = plt.subplot(1,1,1)
psd_RX,f = ax.psd(pim, Fs = FS, Fc = FC_TX, NFFT = 2048, window = np.kaiser(2048,10), noverlap = 1, pad_to = 2048)
psd_NF,f = ax.psd(pred_pim, Fs = FS, Fc = FC_TX, NFFT = 2048, window = np.kaiser(2048,10), noverlap = 1, pad_to = 2048)
ax.set_ylabel(r'PSD, $V^2$/Hz [dB]')
ax.set_xlabel('Frequency, MHz')
ax.set_title('Power spectral density')
plt.show()

### With preliminary applying filter to basis txa basis functions

In [ ]:
A = create_filtered_model_matrix(txa, bias=True)
print(A.shape)
rhs = convolve(rxa, fil['flt_coeff'].flatten())
w = np.linalg.inv(A.conj().T @ A) @ A.conj().T @ rhs
print(w)

In [ ]:
pred_pim = A @ w
filt_signal = rhs - pred_pim

In [ ]:
ax = plt.subplot(1,1,1)
# psd_TX,f = ax.psd(txa, Fs = FS, Fc = FC_TX, NFFT = 2048, window = np.kaiser(2048,10), noverlap = 1, pad_to = 2048)
psd_RX,f = ax.psd(rxa, Fs = FS, Fc = FC_TX, NFFT = 2048, window = np.kaiser(2048,10), noverlap = 1, pad_to = 2048)
# psd_NF,f = ax.psd(pim, Fs = FS, Fc = FC_TX, NFFT = 2048, window = np.kaiser(2048,10), noverlap = 1, pad_to = 2048)
psd_PIM,f = ax.psd(filt_signal, Fs = FS, Fc = FC_TX, NFFT = 2048, window = np.kaiser(2048,10), noverlap = 1, pad_to = 2048)
ax.set_ylabel(r'PSD, $V^2$/Hz [dB]')
ax.set_xlabel('Frequency, MHz')
ax.set_title('Power spectral density')
plt.show()

In [ ]:
ax = plt.subplot(1,1,1)
psd_RX,f = ax.psd(pim, Fs = FS, Fc = FC_TX, NFFT = 2048, window = np.kaiser(2048,10), noverlap = 1, pad_to = 2048)
psd_NF,f = ax.psd(pred_pim, Fs = FS, Fc = FC_TX, NFFT = 2048, window = np.kaiser(2048,10), noverlap = 1, pad_to = 2048)
ax.set_ylabel(r'PSD, $V^2$/Hz [dB]')
ax.set_xlabel('Frequency, MHz')
ax.set_title('Power spectral density')
plt.show()

### Memory channel

In [ ]:
n_pts = 200000
n_back = 40
n_fwd = 10
bf = txa * (np.abs(txa) ** 2)
A = create_shifted_matrix(bf, n_pts, n_back, n_fwd)
print(A.shape)
rhs = rxa[n_back : n_pts + n_back]
print(rhs.shape)
w = np.linalg.inv(A.conj().T @ A) @ A.conj().T @ rhs

In [ ]:
pred_pim = A @ w
filt_signal = rhs - pred_pim

In [ ]:
ax = plt.subplot(1,1,1)
# psd_TX,f = ax.psd(txa, Fs = FS, Fc = FC_TX, NFFT = 2048, window = np.kaiser(2048,10), noverlap = 1, pad_to = 2048)
psd_RX,f = ax.psd(rxa, Fs = FS, Fc = FC_TX, NFFT = 2048, window = np.kaiser(2048,10), noverlap = 1, pad_to = 2048)
# psd_NF,f = ax.psd(pred_pim, Fs = FS, Fc = FC_TX, NFFT = 2048, window = np.kaiser(2048,10), noverlap = 1, pad_to = 2048)
psd_PIM,f = ax.psd(filt_signal, Fs = FS, Fc = FC_TX, NFFT = 2048, window = np.kaiser(2048,10), noverlap = 1, pad_to = 2048)
ax.set_ylabel(r'PSD, $V^2$/Hz [dB]')
ax.set_xlabel('Frequency, MHz')
ax.set_title('Power spectral density')
plt.show()

In [ ]:
ax = plt.subplot(1,1,1)
psd_RX,f = ax.psd(pim, Fs = FS, Fc = FC_TX, NFFT = 2048, window = np.kaiser(2048,10), noverlap = 1, pad_to = 2048)
psd_NF,f = ax.psd(pred_pim, Fs = FS, Fc = FC_TX, NFFT = 2048, window = np.kaiser(2048,10), noverlap = 1, pad_to = 2048)
ax.set_ylabel(r'PSD, $V^2$/Hz [dB]')
ax.set_xlabel('Frequency, MHz')
ax.set_title('Power spectral density')
plt.show()